This tutorial describes how to evaluate new rules applicable to the derivatives data.

In [1]:
from arelle import ModelManager, Cntlr, ModelFormulaObject, ModelXbrl, ViewFileFormulae, XbrlConst, ViewFileRenderedGrid
from arelle import RenderingEvaluator 

In [2]:
import pandas as pd
import numpy as np
from os import listdir
from os.path import join, isfile
import re
from src import Evaluator
import logging
import data_patterns
import pickle

In [3]:
DECIMALS = 0
RULES_PATH = join('..', 'solvency2-rules')
INSTANCES_DATA_PATH = join('..','data','instances')
TEST_DATA_PATH = join('..', 'tests', 'data', 'demo')
FILENAME_DATAPOINTS = 'QRS.csv'
RESULTS_PATH = join('..', 'results')
DATA_PATH = join('..', 'data')
logging.basicConfig(filename = join(RESULTS_PATH, 'rules.log'),level = logging.INFO, 
                    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')

#### Load dictionaries to convert Solvency II rules to FTK rules

In [4]:
def load_obj(name ):
    with open(join('..','data','dicts','') + name + '.pkl', 'rb') as f:
        return pickle.load(f)

d_06 = load_obj('dictionary_s06')
d_08 = load_obj('dictionary_s08')

### Import rules

We start with importing the new rules applicable to the assets and derivatives data. There are several sets of rules applicable to different templates:
* S.06.02.01 / K208A (Information on positions held)
* S.06.02.02 / K208B (Information on assets)
* S.06.02.01 / K208A (Information on positions held) and S.06.02.02 / K208B (Information on assets)
* S.08.01.01.01 / K210A (Information on positions held) and S2.08.01.01.02 / K210B (Information on derivatives)
* S.08.01.01.02 / K210B (Information on derivatives)
Subsequently, we adjust the rules such that they have the right FTK labels.  

#### S06

In [5]:
dfr_s06 = pd.read_excel(join(RULES_PATH,'S2_06_02.xlsx'))
dfr_s06_2 = pd.read_excel(join(RULES_PATH,'S2_06_02_01_02.xlsx'))
dfr_s06_1 = pd.read_excel(join(RULES_PATH,'S2_06_02_01_01.xlsx'))

#Capitalize row-column references:
column_replace = set([column for sublist in [row for row in dfr_s06['pandas ex'].str.findall(r'c\d\d\d\d')] for column in sublist])
for ref in column_replace:
    dfr_s06.replace(to_replace=ref, value=ref.capitalize(), inplace=True, regex=True)
column_replace = set([column for sublist in [row for row in dfr_s06_2['pandas ex'].str.findall(r'c\d\d\d\d')] for column in sublist])
for ref in column_replace:
    dfr_s06_2.replace(to_replace=ref, value=ref.capitalize(), inplace=True, regex=True)
column_replace = set([column for sublist in [row for row in dfr_s06_1['pandas ex'].str.findall(r'c\d\d\d\d')] for column in sublist])
for ref in column_replace:
    dfr_s06_1.replace(to_replace=ref, value=ref.capitalize(), inplace=True, regex=True)
    
#Adjust SII labels to FTK labels
for key in d_06.keys():
    dfr_s06.replace(key, d_06[key], inplace = True,regex=True)
    dfr_s06_1.replace(key, d_06[key], inplace = True,regex=True)
    dfr_s06_2.replace(key, d_06[key], inplace = True,regex=True)
    
# delete rules that do not have cell in FTK
dfr_s06_1 = dfr_s06_1[~dfr_s06_1["pattern_def"].str.contains('S.06.02.01.01')]
dfr_s06_2 = dfr_s06_2[~dfr_s06_2["pattern_def"].str.contains('S.06.02.01.02')]
dfr_s06 = dfr_s06[~dfr_s06["pattern_def"].str.contains('S.06.02.01')]    

#### S08

In [6]:
dfr_s08 = pd.read_excel(join(RULES_PATH,'S2_08_01_01.xlsx'))
dfr_s08_2 = pd.read_excel(join(RULES_PATH,'S2_08_01_01_02.xlsx'))

#Capitalize row-column references:
column_replace = set([column for sublist in [row for row in dfr_s08['pandas ex'].str.findall(r'c\d\d\d\d')] for column in sublist])
for ref in column_replace:
    dfr_s08.replace(to_replace=ref, value=ref.capitalize(), inplace=True, regex=True)
column_replace = set([column for sublist in [row for row in dfr_s08_2['pandas ex'].str.findall(r'c\d\d\d\d')] for column in sublist])
for ref in column_replace:
    dfr_s08_2.replace(to_replace=ref, value=ref.capitalize(), inplace=True, regex=True)
    
#Adjust SII labels to FTK labels
for key in d_08.keys():
    dfr_s08.replace(key, d_08[key], inplace = True,regex=True)
    dfr_s08_2.replace(key, d_08[key], inplace = True,regex=True)

# delete rules that do not have cell in FTK
dfr_s08 = dfr_s08[~dfr_s08["pattern_def"].str.contains('S.08.01.01')]
dfr_s08_2 = dfr_s08_2[~dfr_s08_2["pattern_def"].str.contains('S.08.01.01.02')]

### Import templates

Next we import the reporting data. In the tutorial 'Convert Solvency 2 XBRL-instances to CSV, HTML and pickles' the XBRL-instances are converted to pickle files per template. The pickle files are written to the data/instances folder. We import these pickle files. We merge dataframes for the sets of rules that are applicable to two templates.

#### S06

In [7]:
df_s06_1 = pd.read_pickle(join(TEST_DATA_PATH,'FTK.K208A.pickle')).reset_index() #Import demo pickles
df_s06_2 = pd.read_pickle(join(TEST_DATA_PATH,'FTK.K208B.pickle')).reset_index() #Import demo pickles
#df_s06_1 = pd.read_pickle(join(join(INSTANCES_DATA_PATH,'DNB-NR_FTK-2019-06_2019-12-31_MOD_FTK-BEL'),'FTK.K208A.pickle')).reset_index()
#df_s06_2 = pd.read_pickle(join(join(INSTANCES_DATA_PATH,'DNB-NR_FTK-2019-06_2019-12-31_MOD_FTK-BEL'), 'FTK.K208B.pickle')).reset_index()
df_s06 = pd.merge(df_s06_1,df_s06_2,how='inner', left_on=['entity', 'period', 'K208A,020'], right_on=['entity','period', 'K208B,130']).set_index(['entity', 'period', 'K208A,020'])
df_s06_2 = df_s06_2.set_index(['entity','period', 'K208B,130'])
df_s06_1 = df_s06_1.set_index(['entity', 'period', 'K208A,020'])
df_s06_2['K208B,130'] = df_s06_2.index.get_level_values(2)
df_s06['K208A,020'] = df_s06.index.get_level_values(2)

#### S08

In [8]:
df_s08_1 = pd.read_pickle(join(TEST_DATA_PATH,'FTK.K210A.pickle')).reset_index() #Import demo pickles
df_s08_2 = pd.read_pickle(join(TEST_DATA_PATH, 'FTK.K210B.pickle')).reset_index() #Import demo pickles
#df_s08_1 = pd.read_pickle(join(join(INSTANCES_DATA_PATH,'DNB-NR_FTK-2019-06_2019-12-31_MOD_FTK-BEL'),'FTK.K210A.pickle')).reset_index()
#df_s08_2 = pd.read_pickle(join(join(INSTANCES_DATA_PATH,'DNB-NR_FTK-2019-06_2019-12-31_MOD_FTK-BEL'), 'FTK.K210B.pickle')).reset_index()
df_s08 = pd.merge(df_s08_1,df_s08_2,how='inner', left_on=['entity', 'period', 'K210A,020'], right_on=['entity', 'period', 'K210B,200']).set_index(['entity', 'period', 'K210A,020'])
df_s08_1 = df_s08_1.set_index(['entity', 'period', 'K210A,020'])
df_s08_2 = df_s08_2.set_index(['entity', 'period', 'K210B,200'])
df_s08_2['K210B,200'] = df_s08_2.index.get_level_values(2)
df_s08['K210A,020'] = df_s08.index.get_level_values(2)

In [14]:
df_s08

,,,"K210A,010","FTK.K210A,C030","FTK.K210A,C040","FTK.K210A,C050","FTK.K210A,C060","FTK.K210A,C070","FTK.K210A,C080","FTK.K210A,C090","FTK.K210A,C100","FTK.K210A,C110",...,"FTK.K210B,C250","FTK.K210B,C260","FTK.K210B,C270","FTK.K210B,C280","FTK.K210B,C290","FTK.K210B,C300","FTK.K210B,C310","FTK.K210B,C320","FTK.K210B,C330","FTK.K210B,C340"
entity,period,"K210A,020",,,,,,,,,,,,,,,,,,,,,
LEGALIDENTIFIER20P32,2018-12-31,Key 9,8,Defined benefit and Defined benefit part of Mixed,Text 38,Micro hedge [MI],0.28,1048000.0,Open,1049000.0,1050000.0,1.0,...,Credit quality step 0,Text 42,Text 43,EUR,Text 44,Text 45,Bankruptcy of the underlying or reference enti...,EUR,EUR,1903-12-31 00:00:00


### Evaluate rules

Now we are ready to evaluate the different sets of rules. First, we construct a PatternMiner-object with the data-patterns package using the rules dataframe. Second, we use the analyze-function to get the results of the rules. We do this for each set of rules separately.

#### S06


In [9]:
miner = data_patterns.PatternMiner(df_patterns=dfr_s06)
results_06 = miner.analyze(df_s06)
results_06

100%|█████████████████████████████████████| 1500/1500 [00:07<00:00, 200.50it/s]


result_type        pattern_id  cluster  \
LEGALIDENTIFIER20P32 2018-12-31 Key 2        False  Total solvency 1        0   
                                Key 2        False  Total solvency 2        0   

                                       support  exceptions  confidence  \
LEGALIDENTIFIER20P32 2018-12-31 Key 2        0           1         0.0   
                                Key 2        0           1         0.0   

                                                                             pattern_def  \
LEGALIDENTIFIER20P32 2018-12-31 Key 2  IF ({"FTK.K208A,C070"} != 0) THEN (ABS({"FTK.K...   
                                Key 2  IF ({"FTK.K208A,C060"} != 0) THEN (ABS({"FTK.K...   

                                         P values  \
LEGALIDENTIFIER20P32 2018-12-31 Key 2  1038000.00   
                                Key 2        0.24   

                                                                                Q values  
LEGALIDENTIFIER20P32 2018-12-31 Key 2  [1041000.0, 1038000.0, 0.0315, 1040000.0, 1041...  
                                Key 2  [1041000.0, 0.24, 1042000.0, 1040000.0, 104100...

In [10]:
miner = data_patterns.PatternMiner(df_patterns=dfr_s06_2)
results_06_2 = miner.analyze(df_s06_2)
results_06_2

100%|███████████████████████████████████| 11991/11991 [01:10<00:00, 171.02it/s]


result_type       pattern_id  cluster  \
LEGALIDENTIFIER20P32 2018-12-31 Key 2        False  Unit Solvency 1        0   
                                Key 2        False  Unit Solvency 2        0   

                                       support  exceptions  confidence  \
LEGALIDENTIFIER20P32 2018-12-31 Key 2        0           1         0.0   
                                Key 2        0           1         0.0   

                                                                             pattern_def  \
LEGALIDENTIFIER20P32 2018-12-31 Key 2  IF ({"FTK.K208B,C280"} != 0) THEN ({"FTK.K208B...   
                                Key 2  IF ({"FTK.K208B,C290"} != 0) THEN ({"FTK.K208B...   

                                           P values      Q values  
LEGALIDENTIFIER20P32 2018-12-31 Key 2  1.042000e+06  3.150000e-02  
                                Key 2  3.150000e-02  1.042000e+06

In [11]:
miner = data_patterns.PatternMiner(df_patterns=dfr_s06_1)
results_06_1 = miner.analyze(df_s06_1)
results_06_1

100%|████████████████████████████████████████████| 2/2 [00:00<00:00, 90.97it/s]


result_type pattern_id  cluster  \
LEGALIDENTIFIER20P32 2018-12-31 Key 2        False        Par        0   
                                Key 2        False   Quantity        0   

                                       support  exceptions  confidence  \
LEGALIDENTIFIER20P32 2018-12-31 Key 2        0           1         0.0   
                                Key 2        0           1         0.0   

                                                                             pattern_def  \
LEGALIDENTIFIER20P32 2018-12-31 Key 2  IF ({"FTK.K208A,C060"} != 0) THEN ({"FTK.K208A...   
                                Key 2  IF ({"FTK.K208A,C070"} != 0) THEN ({"FTK.K208A...   

                                         P values    Q values  
LEGALIDENTIFIER20P32 2018-12-31 Key 2        0.24  1038000.00  
                                Key 2  1038000.00        0.24

#### S08

In [16]:
miner = data_patterns.PatternMiner(df_patterns=dfr_s08)
results_08 = miner.analyze(df_s08)
results_08

100%|███████████████████████████████████████| 126/126 [00:00<00:00, 234.77it/s]


,result_type,pattern_id,cluster,support,exceptions,confidence,pattern_def,P values,Q values
index,,,,,,,,,


In [13]:
miner2 = data_patterns.PatternMiner(df_patterns=dfr_s08_2)
results2_08_2 = miner2.analyze(df_s08_2)
results2_08_2

100%|███████████████████████████████████████| 568/568 [00:03<00:00, 188.35it/s]


,result_type,pattern_id,cluster,support,exceptions,confidence,pattern_def,P values,Q values
index,,,,,,,,,
